In [4]:
mari, juksemari = Contributor.objects.filter(display_name__icontains='Haanshus')

In [ ]:
from fuzzywuzzy import process
from django.db.models import Count
contributors = Contributor.objects.annotate(
        num_bylines=Count('byline')
    ).order_by('-num_bylines')
everyone = Contributor.objects.values_list('display_name', flat=True)
for contributor in contributors[:100]:
    best = process.extract(contributor.display_name, everyone, limit=3)
    print(best)
    

In [1]:
from django.db.models import Count

dupes = ImageFile.objects.values('source_file').order_by().annotate(Count('source_file')).filter(source_file__count__gt=1)
dupe_files = [d['source_file'] for d in dupes]
for fn in dupe_files:
    qs = ImageFile.objects.filter(source_file=fn)
    print()
    for img in qs:
        print('{id} {_md5} {_size:>8} {full_width:>4}x{full_height:<4} {source_file}'.format(**img.__dict__))
    


41234 93584a46de990cd257be0da2550ef6ae   105476 1500x950  2017/7/hedvig.jpg
41328 f601e969ea344acaefe7d95aef0b1b19   356760 1610x1020 2017/7/hedvig.jpg

41547 28e486422a95f68a4c3ce02b453aa894  2357490 2500x1667 2017/8/08-NYH-thomasevensen-AN-1.jpg
41538 78c71654c69de46f0268de3e7d976fdd   222192 1500x1000 2017/8/08-NYH-thomasevensen-AN-1.jpg

40204 3311598eb0e2eeac28f79887901863a1   176641 1500x964  2017/6/03-NYH-graver-AN-1-2.jpg
40848 ca4c0f6fe7c7a10f6a290f40841dfbd3  3291804 2500x1606 2017/6/03-NYH-graver-AN-1-2.jpg

41491 88db53d265c19c16b002f2e70087982b   285852 1500x1000 2017/8/08-NYH-DNSoppslag-SN-04.jpg
41546 43a3faa1bd53cdaa4102b96acff18d96  2202531 2500x1667 2017/8/08-NYH-DNSoppslag-SN-04.jpg

41012 5a7afd8b4afafae42aa5cc87f10cbcb4   130055 1500x1000 2017/6/07-NYH-rektor-AN-1-2.jpg
41051 c80a853bee20dffa8872a98ecb8a893c 13697750 5616x3744 2017/6/07-NYH-rektor-AN-1-2.jpg

40148 6307d8b36234c68d357bf914b1666ff2   150930 1500x1000 2017/3/WIEL3303-kopi.jpg
40197 0c851682a2fea89dd

In [19]:
im = ImageFile.objects.first()
im.thumb()

'/media/thumb-cache/9b/e9/9be94901f07f0adcac85534610d1c7ff/Hanna-Skotheim.jpg'

In [15]:
import cv2, numpy

def mse(imageA, imageB):
    # the 'Mean Squared Error' between the two images is the
    # sum of the squared difference between the two images;
    # NOTE: the two images must have the same dimension
    err = numpy.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    
    # return the MSE, the lower the error, the more "similar"
    # the two images are
    return err

def cv_img(instance):
    with instance.source_file as fp:
        _source = fp.read()
    _data = numpy.fromstring(_source, numpy.uint8)
    _image = cv2.imdecode(_data, cv2.IMREAD_COLOR)
    return cv2.resize(
        _image, (97,97), interpolation = cv2.INTER_AREA)

im0 = cv_img(ImageFile.objects.first())
for fn in dupe_files:
    im1, im2 = ImageFile.objects.filter(source_file=fn)[:2]
    diff = mse(cv_img(im1), cv_img(im2))
    print(diff)
    


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [21]:
!pip install --user imagehash

  Using cached PyWavelets-0.5.2-cp36-cp36m-manylinux1_x86_64.whl
  Using cached scipy-0.19.0-cp36-cp36m-manylinux1_x86_64.whl


In [22]:
from PIL import Image
import imagehash

ModuleNotFoundError: No module named 'imagehash'

In [49]:
from django.db import transaction
from django.apps import apps
from django.contrib.contenttypes.fields import GenericForeignKey
from django.db.models.fields.related import ManyToManyField

@transaction.atomic()
def merge(primary_object, *alias_objects):
    """Merge several model instances into one, the `primary_object`.
    Use this function to merge model objects and migrate all of the related
    fields from the alias objects the primary object.
    Usage:
        from django.contrib.auth.models import User
        primary_user = User.objects.get(email='good@example.com')
        duplicate_user = User.objects.get(email='good+duplicate@example.com')
        merge(primary_user, duplicate_user)
    Based on: https://djangosnippets.org/snippets/382/
    """
    generic_fields = get_generic_fields()

    # get related fields
    many_to_many_fields, related_fields = discrimine(
        lambda field: isinstance(field, ManyToManyField),
        primary_object._meta._get_fields(forward=False, include_hidden=True)
    )

    # Loop through all alias objects and migrate their references to the
    # primary object
    for alias_object in alias_objects:
        # Migrate all foreign key references from alias object to primary
        # object.
        for related_object in related_fields:
            # The variable name on the alias_object model.
            alias_varname = related_object.get_accessor_name()
            # The variable name on the related model.
            obj_varname = related_object.field.name
            related_objects = getattr(alias_object, alias_varname)
            for obj in related_objects.all():
                setattr(obj, obj_varname, primary_object)
                obj.save()

        # Migrate all many to many references from alias object to primary
        # object.
        for related_many_object in many_to_many_fields:
            alias_varname = related_many_object.get_accessor_name()
            obj_varname = related_many_object.field.name
            related_many_objects = getattr(alias_object, alias_varname)
            for obj in related_many_objects.all():
                getattr(obj, obj_varname).remove(alias_object)
                getattr(obj, obj_varname).add(primary_object)

        # Migrate all generic foreign key references from alias object to
        # primary object.
        for field in generic_fields:
            filter_kwargs = {}
            filter_kwargs[field.fk_field] = alias_object._get_pk_val()
            filter_kwargs[field.ct_field] = field.get_content_type(alias_object)
            related_objects = field.model.objects.filter(**filter_kwargs)
            for generic_related_object in related_objects:
                setattr(generic_related_object, field.name, primary_object)
                generic_related_object.save()

        if alias_object.id:
            alias_object.delete()

    return primary_object


def get_generic_fields():
    """Return a list of all GenericForeignKeys in all models."""
    generic_fields = []
    for model in apps.get_models():
        for field_name, field in model.__dict__.items():
            if isinstance(field, GenericForeignKey):
                generic_fields.append(field)
    return generic_fields


def discrimine(pred, sequence):
    """Split a collection in two collections using a predicate.
    >>> discrimine(lambda x: x < 5, [3, 4, 5, 6, 7, 8])
    ... ([3, 4], [5, 6, 7, 8])
    """
    positive, negative = [], []
    for item in sequence:
        if pred(item):
            positive.append(item)
        else:
            negative.append(item)
    return positive, negative

for duplicates in dupes:
    for img in duplicates:
        print(img.id, img.md5, img.source_file)
    merge(*duplicates)


41613 45f7c1e7fb2e682016d2914277e4f426 2017/9/09-KUL-Islamsatsing-DK-01.jpg
None 45f7c1e7fb2e682016d2914277e4f426 2017/9/IMG-46162000.jpg
None 45f7c1e7fb2e682016d2914277e4f426 2017/11/09-KUL-Islamsatsing-DK-01.jpg
42174 7636695661fd55779f21bde6ecf55842 2017/11/11-NYH-stolengraver-AN.jpg
42202 7636695661fd55779f21bde6ecf55842 2017/11/11-NYH-stolengraver-AN.jpg
42201 7636695661fd55779f21bde6ecf55842 2017/11/11-NYH-stolengraver-AN.jpg
40484 0168e8815d82b0b15534983d58b7f6c9 2017/5/04-NETT-Idrett-SGS.jpg
40489 0168e8815d82b0b15534983d58b7f6c9 2017/5/04-NETT-Idrett-SGS.jpg
40490 0168e8815d82b0b15534983d58b7f6c9 2017/5/04-NETT-Idrett-SGS.jpg
40522 0168e8815d82b0b15534983d58b7f6c9 2017/5/04-NETT-Idrett-SGS.jpg
40523 0168e8815d82b0b15534983d58b7f6c9 2017/5/04-NETT-Idrett-SGS.jpg
41221 24c8f8123666194ccd854bd67c1fbfcd 2017/7/07-KUL-skitur-GO-23.jpg
41350 24c8f8123666194ccd854bd67c1fbfcd 2017/8/07-KUL-skitur-GO-23.jpg
40914 e381b7b06fa06444f6531e662d5bf409 2017/6/06-NYH-HH-17.jpg
40932 e381b7b06f

41769 92bd25ab75272778a443003263307104 2017/9/Kropp-007-HB.jpg
41858 92bd25ab75272778a443003263307104 2017/9/Kropp-007-HB.jpg
41843 92bd25ab75272778a443003263307104 2017/9/Kropp-007-HB.jpg
41150 8b19dc854c5c30c8caa3121e96c206ea 2017/7/Curt-Rise-2.jpg
41161 8b19dc854c5c30c8caa3121e96c206ea 2017/7/07-NYH-CurtRice-AS-01.jpg
41365 8b19dc854c5c30c8caa3121e96c206ea 2017/8/07-NYH-CurtRice-AS-01.jpg
40356 62ff4a8f61efb0750fea2e17f679f194 2017/4/04-Westerdalsbygg-SGS8.jpg
40337 62ff4a8f61efb0750fea2e17f679f194 2017/4/sj-0167102222-30232932146-o.jpg
40541 f56fb78ea9de82333c5f582b59c4889d 2017/5/05-KUL-gateververe-hh-8.jpg
40719 f56fb78ea9de82333c5f582b59c4889d 2017/6/05-KUL-gateververe-hh-8.jpg
40813 ec342267de7342df50c1868137a44273 2017/6/06-KUL-dagnyartist-NW-2.jpg
40795 ec342267de7342df50c1868137a44273 2017/6/KUL-6-MUSIKK-DAGNY-1-NW.jpg
40931 5aa5de6f9a1fd70869e493d43ba97d8a 2017/6/06-NYH-OSI-MarianneP-HH-07.jpg
40913 5aa5de6f9a1fd70869e493d43ba97d8a 2017/6/06-NYH-HH-16.jpg
39987 2caa80dd63fb

41880 96a24f69de2104ef7faea9445c08679c 2017/10/01-NYH-STUDENTERMEDBARN-AC5.jpg
41883 96a24f69de2104ef7faea9445c08679c 2017/10/01-NYH-STUDENTERMEDBARN-AC5.jpg
40907 55a76f920f0ec8123317ae795e98992b 2017/6/06-NYH-HH-10.jpg
40925 55a76f920f0ec8123317ae795e98992b 2017/6/06-NYH-OSI-MarianneP-HH-01.jpg
40763 feca240f2fc751edf4c0df4dca10147f 2017/6/06-MAG-Babysang-XP.jpg
40754 feca240f2fc751edf4c0df4dca10147f 2017/6/IMG-1615.jpg
41346 feca240f2fc751edf4c0df4dca10147f 2017/8/06-MAG-Babysang-XP.jpg
40790 fb06588b501be992c61825a483a45fd0 2017/6/KUL-6-MUSIKK-DAGNY-5-NW.jpg
40820 fb06588b501be992c61825a483a45fd0 2017/6/06-KUL-dagnyartist-NW-6.jpg
40418 37dca5204e81aa9b209d0e627d0de7f2 2017/4/04-KUL-boehler-DK-2.jpg
40440 37dca5204e81aa9b209d0e627d0de7f2 2017/4/04-KUL-boehler-DK-2.jpg
41055 fc77485ca78cb31ed5ade80531455458 2017/6/Screenshot-2017-02-12-15-45-29.png
41056 fc77485ca78cb31ed5ade80531455458 2017/6/Screenshot-2017-02-12-15-45-29.png
40530 583b75b38fd837f5a635897037b0667c 2017/5/Signe-Ros

40804 1dc4baacc0c83cbab7edf49adf740153 2017/6/KUL-6-MUSIKK-NLA-3-NW.jpg
40850 1dc4baacc0c83cbab7edf49adf740153 2017/6/0KUL-6-MUSIKK-NLA-3-NW.jpg
41808 8c07e5ce3f49510829715129c4ad3869 2017/9/09-NYH-Jus-DK-13.jpg
41838 8c07e5ce3f49510829715129c4ad3869 2017/9/09-NYH-Jus-DK-13.jpg
40067 ec3a09b39ca5fa5f205185c67d4b4d30 2017/3/netturix10.jpg
40072 ec3a09b39ca5fa5f205185c67d4b4d30 2017/3/netturix10.jpg
40235 93cb1a8df0ea8787f6683b887c24e0d6 2017/4/16326022-10209931501677411-1878425910-o.jpg
40234 93cb1a8df0ea8787f6683b887c24e0d6 2017/4/16326022-10209931501677411-1878425910-o.jpg
41616 894e50f31a97d9acb51b2f12695159fa 2017/9/09-KUL-Islamsatsing-DK-04.jpg
41602 894e50f31a97d9acb51b2f12695159fa 2017/9/IMG-46862000.jpg
41606 23ef57358a5681b39fb95fed6466bee5 2017/9/09-KUL-min-studietid-Brundtland-DK-01.jpg
41592 23ef57358a5681b39fb95fed6466bee5 2017/9/IMG-45412000.jpg
41878 23ef57358a5681b39fb95fed6466bee5 2017/10/09-KUL-min-studietid-Brundtland-DK-01.jpg
41774 91fd5321d036b98478dbcc18f83876a1 2

41104 add8f0bd32d44ba4cafd54144e84e046 2017/7/07-KUL-dybdahl-AN-4.jpg
40845 add8f0bd32d44ba4cafd54144e84e046 2017/6/07-KUL-dybdahl-AN-4.jpg
42198 ced8517344870a99c430275c4617385b 2017/11/Tanjafrontbilde.jpg
42221 ced8517344870a99c430275c4617385b 2017/11/Tanjafrontbilde.jpg
41628 e2d4f47d636d8acb7bb75bd05557fc97 2017/9/unspecified-1.jpg
41591 e2d4f47d636d8acb7bb75bd05557fc97 2017/9/unspecified-1.jpg
41640 e2d4f47d636d8acb7bb75bd05557fc97 2017/9/unspecified-1.jpg
41639 e2d4f47d636d8acb7bb75bd05557fc97 2017/9/unspecified-1.jpg
40865 b43e82f86aca66bcb1c49a7e09986c80 2017/6/06-NYH-HH-32.jpg
40883 b43e82f86aca66bcb1c49a7e09986c80 2017/6/06-NYH-sensur-nf-HH-3.jpg
40422 55372a718021d2fd2ea63fd93c4dc072 2017/4/miller-world-physsssical-wall-mural-lg.jpg
40441 55372a718021d2fd2ea63fd93c4dc072 2017/4/miller-world-physsssical-wall-mural-lg.jpg
40309 db58f0e2ac9b93842fb883842b26282b 2017/4/alvin-ailey-29.jpg
40454 db58f0e2ac9b93842fb883842b26282b 2017/4/alvin-ailey-29.jpg
40895 b40a577e7f0906bf7a4d7

41623 7d679220f83218373dbc9bfd4df1c3ac 2017/9/unspecified-3.jpg
41629 7d679220f83218373dbc9bfd4df1c3ac 2017/9/unspecified-3.jpg
39906 94c91395bdc53c75ef63fe481ea68a22 2017/2/02-URIX-studengjeld-DK-4.jpg
40056 94c91395bdc53c75ef63fe481ea68a22 2017/3/02-URIX-studengjeld-DK-4.jpg
41050 0ee27ff27a689c5e7bf0ac28b73c1bd0 2017/6/07-OYE-debatt-AN-1.jpg
41034 0ee27ff27a689c5e7bf0ac28b73c1bd0 2017/6/07-OYE-debatt-AN-1.jpg
41041 0ee27ff27a689c5e7bf0ac28b73c1bd0 2017/6/07-OYE-debatt-AN-1.jpg
40015 0779aaaa4f858ab1c7072b3f836f0652 2017/2/02-NYH-westerdals-SOE-4.jpg
40014 0779aaaa4f858ab1c7072b3f836f0652 2017/2/02-NYH-westerdals-SOE-4.jpg
39963 0779aaaa4f858ab1c7072b3f836f0652 2017/2/02-NYH-westerdals-SOE-4.jpg
39998 daab5ecd816b1111b3476c71aab165e3 2017/2/02-NYH-kvinnekveld-SOE-3web.jpg
39995 daab5ecd816b1111b3476c71aab165e3 2017/2/02-NYH-kvinnekveld-SOE-3web.jpg
40796 a4126e90b7ce9e4b711e4eeb1bda876d 2017/6/KUL-6-MUSIKK-DAGNY-10-NW.jpg
40810 a4126e90b7ce9e4b711e4eeb1bda876d 2017/6/06-KUL-dagnyarti

40782 65c6228de00cc7098968e53a2d9532d6 2017/6/05-ANM-johndee-VIB-4.jpg
40775 65c6228de00cc7098968e53a2d9532d6 2017/6/VildeBorse-JohnDee003.jpg
40698 8efa2b15f32e1d1bc9a3a927eba634eb 2017/5/trynebildet.jpg
41040 8efa2b15f32e1d1bc9a3a927eba634eb 2017/6/trynebildet.jpg
40702 0b3d442332227e03dee5272a782de2d6 2017/5/05-NYH-prorektor-HMLS-5.jpg
40703 0b3d442332227e03dee5272a782de2d6 2017/5/05-NYH-prorektor-HMLS-5.jpg
42066 3d2af9e41781ff42df7867c20da53f3f 2017/11/XX-MAG-sexekteskap-EBJ-22.jpg
42217 3d2af9e41781ff42df7867c20da53f3f 2017/11/XX-MAG-sexekteskap-EBJ-22.jpg
40755 adb6b7107b3da28f42ae7c3576786abf 2017/6/IMG-1655.jpg
40767 adb6b7107b3da28f42ae7c3576786abf 2017/6/06-MAG-HelgaByfuglien-XP-3.jpg
41307 c417007e1086980231c0f8d5a431231a 2017/7/07-NYH-hcpauls-AN-04.jpg
41483 c417007e1086980231c0f8d5a431231a 2017/8/07-NYH-hcpauls-AN-04.jpg
41323 c417007e1086980231c0f8d5a431231a 2017/7/07-NYH-hcpauls-AN-04.jpg
41453 b848a611653a091466206543bed1299b 2017/9/09-KUL-Studentforeninger-Aas-DK-12.j

41611 07c33adc857ea333581214fb7778a1a0 2017/9/09-KUL-min-studietid-Brundtland-DK-06.jpg
41597 07c33adc857ea333581214fb7778a1a0 2017/9/IMG-45932000.jpg
40779 b175688e487c934c701c3056477a3608 2017/6/05-ANM-johndee-VIB-1.jpg
40772 b175688e487c934c701c3056477a3608 2017/6/VildeBorse-JohnDee000.jpg
40345 f55326858a384864d5a38a5147a54818 2017/4/04-Westerdalsbygg-SGS1-1.jpg
40340 f55326858a384864d5a38a5147a54818 2017/4/sj-0167132222-30267774275-o.jpg
40477 012fe2315f618ad78e7589994aff0242 2017/5/13-NYH-nyhlaringsmi-01-KB.jpg
40478 012fe2315f618ad78e7589994aff0242 2017/5/13-NYH-nyhlaringsmi-01-KB.jpg
40483 012fe2315f618ad78e7589994aff0242 2017/5/13-NYH-nyhlaringsmi-01-KB.jpg
40177 c0429940578a0345bd7f3ca12abcae53 2017/3/MG-9812.jpg
40210 c0429940578a0345bd7f3ca12abcae53 2017/3/MG-9812.jpg
40208 c0429940578a0345bd7f3ca12abcae53 2017/3/MG-9812.jpg
40113 9c3027eb9f2bd1686aaf8b4a12935093 2017/3/0ThomasThwaites0-PhotoCredit-Tim-Bowditch.jpg
40216 9c3027eb9f2bd1686aaf8b4a12935093 2017/4/0ThomasThwait